In [37]:
import torch
import numpy as np
from torch.distributions import MultivariateNormal
from torch import nn
import torch.nn.functional as F
import gymnasium as gym
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter


In [7]:
import time

In [8]:
class neural_network(nn.Module):
    def __init__(self,input_dims,output_dims):
        super(neural_network,self).__init__()
        self.layer1=nn.Linear(input_dims,64)
        self.layer2=nn.Linear(64,64)
        self.layer3=nn.Linear(64,output_dims)
    def forward(self,state):
        if isinstance(state,np.ndarray):
            state=torch.tensor(state,dtype=torch.float)
            
        act1=F.relu(self.layer1(state))
        act2=F.relu(self.layer2(act1))
        output=self.layer3(act2)
        return output
    

In [6]:
env1=gym.make('Pendulum-v1')
# env1=gym.vector.SyncVectorEnv(env)
env1.reset()


(array([-0.83772224, -0.5460965 ,  0.89465517], dtype=float32), {})

In [8]:
L=[]
for i in range(4):
    
    L.append(env.observation_space.sample())

In [13]:
new_stata,reward,done,_,_=env.step(np.array([1,0,0,0]))

AssertionError: array([1, 0, 0, 0]) (<class 'numpy.ndarray'>) invalid

In [ ]:
mean=
dist = MultivariateNormal(mean, a)

In [11]:
actor=neural_network(4,4)
actor(np.array([-2.6352916e+00, -2.2844338e+38,  2.9501456e-01, -2.7980816e+37],
       dtype=float)).detach().numpy()

array([ 1.0739300e+37,  1.4293945e+37,  1.0251376e+36, -5.9085093e+36],
      dtype=float32)

In [12]:
actor.state_dict(),

(OrderedDict([('layer1.weight',
               tensor([[-0.4751, -0.4852,  0.1991, -0.3985],
                       [ 0.0364, -0.1829, -0.4313,  0.0102],
                       [-0.4279, -0.4596, -0.4668,  0.1448],
                       [ 0.0063,  0.3088, -0.0284, -0.3186],
                       [ 0.2166,  0.4812, -0.4225,  0.2312],
                       [ 0.4791, -0.0249,  0.0465, -0.2635],
                       [ 0.4325, -0.3278,  0.1359,  0.0443],
                       [-0.0836, -0.3002, -0.4430, -0.0476],
                       [ 0.3805,  0.0448, -0.2599,  0.2697],
                       [-0.0776,  0.4100, -0.0113,  0.3646],
                       [-0.3142, -0.3397, -0.2897, -0.1162],
                       [-0.4083,  0.3879,  0.3437, -0.4438],
                       [-0.4216, -0.1533, -0.0685, -0.2068],
                       [-0.2390, -0.4049, -0.2300,  0.0419],
                       [ 0.3623, -0.0320, -0.3449, -0.1044],
                       [-0.4619,  0.3191,  0.4359, -0

In [31]:
env.step(1)[0]

array([-0.01018491,  0.42836714, -0.01215711, -0.63799953], dtype=float32)

In [103]:
class PPO_version0:
    def __init__(self,env,max_timesteps,epochs,max_iteration_episodes,gamma,clip,lr,device,num_envs,writer,minibatch_size):
        self.env=env
        self.actor=neural_network(self.env.single_observation_space.shape[0],self.env.single_action_space.shape[0]).to(device)
        self.cretic=neural_network(self.env.single_observation_space.shape[0],1).to(device)
        self.max_timesteps=max_timesteps
        self.epochs=epochs
        self.max_iteration_episodes=max_iteration_episodes
        self.gamma=gamma
        self.clip=clip
        self.lr=lr
        self.actor_optimization= Adam(self.actor.parameters() , lr=self.lr)
        self.critic_optimization= Adam(self.cretic.parameters() , lr=self.lr)
        self.cov_var=torch.full(size=(self.env.single_action_space.shape[0],),fill_value=0.5).to(device)
        self.cov_mat=torch.diag(self.cov_var).to(device)
        self.device=device
        self.num_envs=num_envs
        self.global_step=0
        self.minibatch_size=minibatch_size
        self.writer=writer
    def get_action(self,state):
        mean = self.actor(state)
        dist = MultivariateNormal(mean, self.cov_mat)
        action=dist.sample()
    
        log_probs = dist.log_prob(action)
 
        return action, log_probs
        
        
    def train(self):
        
        t=0
        batch_states = torch.zeros((self.max_timesteps, self.num_envs) + self.env.single_observation_space.shape).to(self.device)
        batch_actions = torch.zeros((self.max_timesteps, self.num_envs) + self.env.single_action_space.shape).to(self.device)
        batch_rew = torch.zeros((self.max_timesteps, self.num_envs)).to(self.device)
        batch_done = torch.zeros((self.max_timesteps, self.num_envs)).to(self.device)
        batch_values=torch.zeros((self.max_timesteps, self.num_envs)).to(self.device)
        # TRY NOT TO MODIFY: start the game
        state,_=self.env.reset()
        next_obs = torch.Tensor(state).to(self.device)
        next_done = torch.zeros(self.num_envs).to(self.device)
        
            
        ep_lenght=0
        ep_rew=np.zeros((self.num_envs))
        for i in range(self.max_iteration_episodes):
            self.global_step+=1
            batch_states[i]=next_obs
            V=self.evaluate_V(next_obs).flatten()
            batch_values[i]=V
            batch_done[i]=next_done
            
            action,_=self.get_action(next_obs)
            
            next_obs,reward,done,_,info=self.env.step(action.cpu().numpy())
            ep_rew+=reward
            
            next_obs, next_done = torch.Tensor(next_obs).to(self.device), torch.Tensor(done).to(self.device)
            batch_actions[i]=action
            batch_rew[i] = torch.tensor(reward).to(self.device).view(-1)
            # for item in info:
            #     if "episode" in item.keys():
            #         print(f"global_step={t}, episodic_return={item['episode']['r']}")
            #         self.writer.add_scalar("charts/episodic_return", item["episode"]["r"], t)
            #         self.writer.add_scalar("charts/episodic_length", item["episode"]["l"], t)
            #         break
            
            if 1 in done or i==self.max_iteration_episodes -1:
                print(ep_rew[0])
                self.writer.add_scalar('charts/episodic_return',ep_rew[0],self.global_step)
                eps_rew=np.zeros((self.num_envs))
            state=next_obs
            # L=[]
            # L.append(action)
        batch_states = batch_states.reshape((-1,) + self.env.single_observation_space.shape)
        batch_actions = batch_actions.reshape((-1,) + self.env.single_action_space.shape)
        

        return batch_rew,batch_states,batch_actions,next_obs,next_done,batch_done,batch_values
    
    def evaluate_prob(self,batch_states,batch_actions):
        mean=self.actor(batch_states)
        dist=MultivariateNormal(mean,self.cov_mat)
        log_prob=dist.log_prob(batch_actions)
        V=self.cretic(batch_states).squeeze()
        return V,log_prob
    
    def evaluate_V(self,batch_states):
        V=self.cretic(batch_states)
        return V.detach()
    
    def evaluate_Q(self,batch_rew,next_obs,next_done,batch_done):
        
        V=self.evaluate_V(next_obs).reshape(1, -1)
       
        batch_Q = torch.zeros_like(batch_rew).to(self.device)
     
        for j in reversed(range(self.max_timesteps)):
            
            if j==self.max_timesteps - 1:
                nextnonterminal = 1.0 - next_done
                next_return = V
            else:
                nextnonterminal = 1.0 - batch_done[j + 1]
                next_return = batch_rew[j + 1]
        
            
            batch_Q[j] = batch_rew[j] + self.gamma * nextnonterminal * next_return
        return batch_Q.detach()
    
    def scale(self,A):
        return (A-A.mean())/(A.std()+1e-10)
        
   
    def learn(self,x):
        k=0
        while k<x:
            batch_rew,batch_states,batch_actions,next_obs,next_done,batch_done,batch_values=self.train()
            _,batch_log_prob=self.evaluate_prob(batch_states,batch_actions)
            batch_log_prob=batch_log_prob.detach()
            V_k = batch_values.detach()
            Q=self.evaluate_Q(batch_rew,next_obs,next_done,batch_done)
            A_k= Q -V_k
            
            A_k=A_k.reshape(-1)
            batch_size=self.max_timesteps*self.num_envs
            b_inds = np.arange(batch_size)
            

            Q=Q.reshape(-1)
            for epoch in range(self.epochs):
                np.random.shuffle(b_inds)
                for start in range(0, batch_size, self.minibatch_size):
                    end = start + self.minibatch_size
                    mb_inds = b_inds[start:end]
                    V,curr_log_prob=self.evaluate_prob(batch_states[mb_inds],batch_actions[mb_inds])
                    ratio=torch.exp(curr_log_prob - batch_log_prob[mb_inds]).to(self.device)
                    A_km=self.scale(A_k[mb_inds])
                    loss1=ratio*A_km
                    loss2=(torch.clamp(ratio, 1-self.clip ,1+self.clip ).to(self.device)) *A_km
                    loss_pi=(-torch.min(loss1,loss2)).mean()
                    
                    self.actor_optimization.zero_grad()
                    loss_pi.backward()
                    self.actor_optimization.step()
                
                
                    loss_v=nn.MSELoss()(V, Q[mb_inds])
                    self.critic_optimization.zero_grad()
                    loss_v.backward()
                    self.critic_optimization.step()
            k+=Q.shape[0]
    def test(self,episodes):
        env=gym.make('Pendulum-v1',render_mode='human')
        for episode in range(episodes):
            ep_len = 0            # episodic length
            ep_ret = 0            # episodic retur
            done=False
            obs,_=env.reset()
            env.render()
            for i in range(200):

                # Render environment if specified, off by default
                
                obs=torch.tensor(obs, dtype=torch.float).to(self.device)

                # Query deterministic action from policy and run it
                
                action = self.actor(obs).detach().cpu().numpy()
                
                obs, rew, done, _,_ = env.step(action)

                # Sum all episodic rewards as we go along
                ep_ret += rew
                
            # Track episodic length

            # returns episodic length and return in this iteration
                print('reward :',ep_ret)
        env.close()
            
        
                
                
            
        


In [106]:
def make_env(gym_id):
    def thunk():
        env = gym.make(gym_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
      
        return env

    return thunk
envs = gym.vector.SyncVectorEnv(
        [make_env("Pendulum-v1")for i in range(1)])
device = torch.device("cuda")
writer = SummaryWriter("raed8/PPO_version1")

model=PPO_version0(envs,1_000,10,500,0.99,0.2,3e-4,device,1,writer,4)
model.learn(1_000_000)
writer.close()

-2805.7918266604074
-4108.484640684432
-2636.826374505506
-3263.9824725713897
-2977.7692220526114
-3311.4731433883953
-3649.6790973663237
-3427.5252895006774
-4013.941973625276
-3558.1435790632495
-3906.401093875281
-4120.477552564103
-4210.450514183049
-4079.506168951416
-3762.0381515164468
-3996.2298326903606
-4254.276974602877
-3928.0589535118306
-4127.515475835314
-4049.6310945800537
-3933.3666469669615
-3998.0030869080883
-4057.238287447978
-3994.570023893927
-3720.6906389876754
-3756.721738699099
-3724.866697308157
-3696.1716324909044
-3672.946847351104
-3686.4618546729803
-3789.546991797239
-3630.813061802538
-3617.2164741504766
-3590.9858724982228
-3722.9727111015745
-3688.246552546248
-3562.766922067194
-3579.1196060685274
-3520.039722025734
-3804.05751295162
-3740.9799265410097
-3627.445400100638
-3612.1556575993327
-3674.536686192134
-3759.7955523742626
-3236.108044573208
-3689.7740140878873
-3663.365725370989
-3634.1680837188383
-3660.229337316839
-3604.9008555923137
-3598.

In [95]:
model.writer.close()

AttributeError: 'PPO_version0' object has no attribute 'writer'

In [19]:
batch_states = torch.zeros((15, 3)).detach()
n=neural_network(envs.single_observation_space.shape[0],1)
n(batch_states).squeeze().requires_grad


True

In [100]:
batch_states = torch.zeros((15, 3)).detach()

In [101]:
batch_states

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [102]:
batch_states.long()

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [98]:
model.test(5)

reward : -2.8855900563785033
reward : -5.924633123417365
reward : -9.229213241518249
reward : -12.921749766383574
reward : -17.127221707126424
reward : -21.969624751952214
reward : -27.565531507796987
reward : -34.01505136830031
reward : -41.39146560955585
reward : -49.73176037704587
reward : -59.030625473741516
reward : -69.23974906789809
reward : -79.86609383490584
reward : -89.46169045443483
reward : -98.15804413684744
reward : -106.12519067494951
reward : -113.57245531569336
reward : -120.65264535025136
reward : -127.43297525421019
reward : -133.9512890354431
reward : -140.24086925534712
reward : -146.34245749315576
reward : -152.30285472588002
reward : -158.17340407852637
reward : -164.0084154925818
reward : -169.86356137640416
reward : -175.79424538005188
reward : -181.85393267817477
reward : -188.09243510464887
reward : -194.55416844858453
reward : -201.2764368522924
reward : -208.28784019262858
reward : -215.60693123797998
reward : -223.2412576303341
reward : -231.1869021343131

In [ ]:
neural_network(self.env.single_observation_space.shape[0],self.env.single_action_space.shape[0])

In [177]:
writer = SummaryWriter(f"runs/PPO_version0__{int(time.time())}")
for i in range(6):   
    writer.add_scalar('test',i*5,global_step=i)

In [131]:
_,_,_,_,info=envs.step(envs.action_space.sample())

In [182]:
np.zeros((4))

array([0., 0., 0., 0.])

In [85]:
envs.reset()
action=envs.action_space.sample()

a,r,_,_,_=envs.step(action)

In [99]:
torch.tensor(r).view(-1)

NameError: name 'r' is not defined

In [88]:
r

array([-3.79173457, -8.0906354 , -0.00858438, -1.92236239])

In [76]:
np.array(envs.single_observation_space.shape).prod()

3

In [82]:
np.array(envs.single_action_space.shape).prod()

1

In [71]:
envs

SyncVectorEnv(4)

In [15]:
model.test(5)

reward : -5.683198897724099
reward : -11.208939319031344
reward : -16.775036598660108
reward : -22.602178717270895
reward : -28.92496160219052
reward : -35.97766688355183
reward : -43.97362665114854
reward : -53.08937617572663
reward : -63.44965861216113
reward : -74.21500973868085
reward : -83.92743732916831
reward : -92.56446406350878
reward : -100.14580845864471
reward : -106.73886257060795
reward : -112.45662330285907
reward : -117.44923092596498
reward : -121.89401106947278
reward : -125.98452627156624
reward : -129.92013617933108
reward : -133.90201972758118
reward : -138.13371340125065
reward : -142.82158102885188
reward : -148.17749349262945
reward : -154.41592448927324
reward : -161.7440337743586
reward : -170.34196655240683
reward : -180.34314324381728
reward : -191.8178053249521
reward : -202.45591257257533
reward : -211.81543352545953
reward : -219.91888461981281
reward : -226.83907314373178
reward : -232.6979051976398
reward : -237.65808762311315
reward : -241.911972986950

In [17]:
env.observation_space.sample()

array([-0.4086331 ,  0.69766575,  4.6289186 ], dtype=float32)

In [56]:
a=np.arange(15)
np.random.shuffle(a)
print(a)

for i in range(0,15,3):
    end=i+3
    s=a[i:end]
    print(s)
    obs[s]
    print(obs)
    

[ 5 12 11  3  9  7  4  8 14 10  1  2  6  0 13]
[ 5 12 11]
[[0.38666955 0.42141143 0.40985762 0.01468469]
 [0.40550966 0.60388158 0.95610237 0.1183427 ]
 [0.37009791 0.84542052 0.6843701  0.76762253]
 [0.46696066 0.9848017  0.85470799 0.03833604]
 [0.82915049 0.94522254 0.35927148 0.6564547 ]
 [0.78730688 0.14303128 0.09686446 0.13922546]
 [0.56542147 0.64102809 0.34124912 0.970972  ]
 [0.19045474 0.18122974 0.26487631 0.39102986]
 [0.78770073 0.46208566 0.41728664 0.30025914]
 [0.01678557 0.44009727 0.70587051 0.51571985]
 [0.40680957 0.04405536 0.91335008 0.14416954]
 [0.51247322 0.32248927 0.74582712 0.1620112 ]
 [0.88734814 0.2709744  0.50023687 0.22386461]
 [0.28860308 0.956961   0.85164722 0.58042007]
 [0.39329904 0.59037354 0.22291441 0.73320636]]
[3 9 7]
[[0.38666955 0.42141143 0.40985762 0.01468469]
 [0.40550966 0.60388158 0.95610237 0.1183427 ]
 [0.37009791 0.84542052 0.6843701  0.76762253]
 [0.46696066 0.9848017  0.85470799 0.03833604]
 [0.82915049 0.94522254 0.35927148 0.656

In [46]:
obs = np.random.random((15,4))

In [60]:
obs[ [6 , 1 , 9  ]]

array([[0.56542147, 0.64102809, 0.34124912, 0.970972  ],
       [0.40550966, 0.60388158, 0.95610237, 0.1183427 ],
       [0.01678557, 0.44009727, 0.70587051, 0.51571985]])

In [64]:
obs[6]

array([0.56542147, 0.64102809, 0.34124912, 0.970972  ])

In [61]:
obs.shape

(15, 4)

In [37]:
import numpy as np

# Suppose the environment has a single observation space shape of (2, 2)
single_observation_space_shape = (3,)

# Generate a random observation array with shape (4, 2, 2)
obs = np.random.random((12,4,4))

# Reshape the obs array to match the single_observation_space shape
b_obs = obs.reshape((-1,) +(4,))

# Print the original and reshaped arrays

print("\nReshaped b_obs shape:", b_obs.shape)
print("Reshaped b_obs array:")
print(b_obs)



Reshaped b_obs shape: (48, 4)
Reshaped b_obs array:
[[0.69481385 0.55587527 0.59367642 0.43085629]
 [0.06018066 0.55972636 0.50987688 0.7082999 ]
 [0.6687081  0.01696573 0.95235547 0.71893289]
 [0.86241116 0.94448639 0.1919117  0.79956276]
 [0.15974269 0.47357438 0.91224694 0.84041706]
 [0.82476058 0.45363581 0.64457352 0.99012367]
 [0.06977143 0.38832778 0.45637532 0.31244277]
 [0.01991948 0.46190861 0.87037294 0.40705147]
 [0.53147438 0.77282349 0.70829398 0.39447202]
 [0.88396667 0.68114601 0.870091   0.5473923 ]
 [0.50992395 0.3644409  0.46914774 0.15111726]
 [0.20977973 0.36086775 0.87545821 0.32520184]
 [0.40439464 0.39570552 0.3175138  0.26564573]
 [0.74911804 0.73182795 0.07421998 0.38556994]
 [0.68844174 0.88303675 0.4857736  0.97369597]
 [0.86202223 0.90198655 0.20295541 0.65402601]
 [0.79461161 0.06150136 0.01174739 0.19729028]
 [0.22011213 0.81956466 0.25189302 0.49308405]
 [0.94009976 0.52974383 0.91776646 0.9444369 ]
 [0.20540657 0.19019668 0.25355413 0.74045101]
 [0.617

In [124]:
import gym
env=gym.make('CartPole-v1')
env = gym.wrappers.RecordEpisodeStatistics(env)

In [125]:
env.reset()
env.step(1)

C:\Users\raeda\AppData\Roaming\Python\Python311\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


(array([ 0.01414056,  0.20427078, -0.00602352, -0.33814648], dtype=float32),
 1.0,
 False,
 False,
 {})

In [228]:
state,_=env1.reset()

a=torch.tensor(state, dtype=torch.float)
# a=a.flatten()
A=torch.diag(a)

A.shape

torch.Size([3, 3])

In [132]:
a= torch.tensor(env.step(1)[0], dtype=torch.float).to(device)
a

tensor([-0.0430,  0.4156, -0.0049, -0.5542], device='cuda:0')

In [ ]:
'timesteps_per_batch': 2048, 
				'max_timesteps_per_episode': 200, 
				'gamma': 0.99, 
				'n_updates_per_iteration': 10,
				'lr': 3e-4, 
				'clip': 0.2,
				'render': True,
				'render_every_i': 10

In [71]:
env1=gym.make('Pendulum-v1')
env1.action_space.sample()

array([-0.7888304], dtype=float32)

In [55]:
state=env1.reset()
env1.step(env1.action_space.sample())

(array([-0.91116554, -0.41204044,  0.3663817 ], dtype=float32),
 -7.548833920978005,
 False,
 False,
 {})

In [61]:
env1.close()